# Extracción de texto con base en el contexto

## Importar librerías

In [1]:
import pathlib
import re
from functools import partial
from typing import Generator

from bs4 import BeautifulSoup, Doctype, NavigableString, SoupStrainer, Tag
from html2text import HTML2Text
from IPython.core.display import Markdown
from langchain.document_loaders import DocugamiLoader, RecursiveUrlLoader



## Web

### Dataset y función de utilidad

In [2]:
doc_url = "https://python.langchain.com/docs/get_started/quickstart"

load_documents = partial(
    RecursiveUrlLoader,
    url=doc_url,
    max_depth=3,
    prevent_outside=True,
    check_response_status=True,
)

### Extracción de texto sin tener en cuenta el contexto

La primera aproximación para extraer texto de una página web es simplemente obtener el texto de todos los elementos de la página.

In [3]:
def webpage_text_extractor(html: str) -> str:
    return BeautifulSoup(html, "lxml").get_text()


loader = load_documents(
    extractor=webpage_text_extractor,
)

docs_without_data_context = loader.load()
print(docs_without_data_context[0].page_content[:520])






Quickstart | 🦜️🔗 Langchain







Skip to main content🦜️🔗 LangChainDocsUse casesIntegrationsGuidesAPIMoreVersioningChangelogDeveloper's guideTemplatesCookbooksTutorialsYouTube videos🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsChatSearchGet startedIntroductionInstallationQuickstartSecurityLangChain Expression LanguageGet startedWhy use LCELInterfaceStreamingHow toCookbookLangChain Expression Language (LCEL)ModulesModel I/ORetrievalAgentsChainsMoreLangServeLangSmi


### Extracción de texto teniendo un poco de contexto

El texto de la documentación de `Langchain` está escrito en `Markdown` y por lo tanto, tiene una estructura que puede ser aprovechada para extraer el texto de manera más precisa.

Utilicemos una librería que nos permita convertir el texto de `HTML` a `Markdown` y así poder extraer el texto de manera más precisa.

In [4]:
def markdown_extractor(html:str)->str:
    html2text=HTML2Text()
    html2text.ignore_links=False
    html2text.ignore_images=False
    return html2text.handle(html)

loader=load_documents(extractor=markdown_extractor)

docs_with_a_bit_of_context = loader.load()
print(docs_with_a_bit_of_context[0].page_content[:720])

Skip to main content

[ **🦜️🔗 LangChain**](/)[Docs](/docs/get_started/introduction)[Use
cases](/docs/use_cases)[Integrations](/docs/integrations/providers)[Guides](/docs/guides/debugging)[API](https://api.python.langchain.com)

More

  * [Versioning](/docs/packages)
  * [Changelog](/docs/changelog)
  * [Developer's guide](/docs/contributing)
  * [Templates](/docs/templates/)
  * [Cookbooks](https://github.com/langchain-ai/langchain/blob/master/cookbook/README.md)
  * [Tutorials](/docs/additional_resources/tutorials)
  * [YouTube videos](/docs/additional_resources/youtube)

🦜️🔗

  * [LangSmith](https://smith.langchain.com)
  * [LangSmith Docs](https://docs.smith.langchain.com/)
  * [LangServe GitHub](https://git


### Extracción de texto teniendo en cuenta el contexto

Si bien, cuando utilizamos una libreraía para convertir el texto de `HTML` a `Markdown` pudimos extraer el texto de manera más precisa, aún hay algunos casos en los que no se logra extraer el texto de manera correcta.

Es aquí donde entra en juego el dominio del problema. Con base en el conocimiento que tenemos del problema, podemos crear una función que nos permita extraer el texto de manera más precisa.

Imagina que `langchain_docs_extractor` es como un obrero especializado en una fábrica cuyo trabajo es transformar materias primas (documentos HTML) en un producto terminado (un string limpio y formateado). Este obrero usa una herramienta especial, `get_text`, como una máquina para procesar las materias primas en piezas utilizables, examinando cada componente de la materia prima **pieza por pieza**, y usa el mismo proceso repetidamente (**recursividad**) para descomponer los componentes en su forma más simple. Al final, ensambla todas las piezas procesadas en un producto completo y hace algunos refinamientos finales antes de que el producto salga de la fábrica.

In [5]:
def langchain_docs_extractor(html: str,include_output_cells: bool,path_url: str | None = None) -> str:
    soup = BeautifulSoup(html,"lxml",parse_only=SoupStrainer(name="article"))

    # Remove all the tags that are not meaningful for the extraction.
    SCAPE_TAGS = ["nav", "footer", "aside", "script", "style"]
    [tag.decompose() for tag in soup.find_all(SCAPE_TAGS)]

    # get_text() method returns the text of the tag and all its children.
    def get_text(tag: Tag) -> Generator[str, None, None]:
        for child in tag.children:
            if isinstance(child, Doctype):
                continue

            if isinstance(child, NavigableString):
                yield child.get_text()
            elif isinstance(child, Tag):
                if child.name in ["h1", "h2", "h3", "h4", "h5", "h6"]:
                    text = child.get_text(strip=False)

                    if text == "API Reference:":
                        yield f"> **{text}**\n"
                        ul = child.find_next_sibling("ul")
                        if ul is not None and isinstance(ul, Tag):
                            ul.attrs["api_reference"] = "true"
                    else:
                        yield f"{'#' * int(child.name[1:])} "
                        yield from child.get_text(strip=False)

                        if path_url is not None:
                            link = child.find("a")
                            if link is not None:
                                yield f" [](/{path_url}/{link.get('href')})"
                        yield "\n\n"
                elif child.name == "a":
                    yield f"[{child.get_text(strip=False)}]({child.get('href')})"
                elif child.name == "img":
                    yield f"![{child.get('alt', '')}]({child.get('src')})"
                elif child.name in ["strong", "b"]:
                    yield f"**{child.get_text(strip=False)}**"
                elif child.name in ["em", "i"]:
                    yield f"_{child.get_text(strip=False)}_"
                elif child.name == "br":
                    yield "\n"
                elif child.name == "code":
                    parent = child.find_parent()
                    if parent is not None and parent.name == "pre":
                        classes = parent.attrs.get("class", "")

                        language = next(
                            filter(lambda x: re.match(r"language-\w+", x), classes),
                            None,
                        )
                        if language is None:
                            language = ""
                        else:
                            language = language.split("-")[1]

                        if language in ["pycon", "text"] and not include_output_cells:
                            continue

                        lines: list[str] = []
                        for span in child.find_all("span", class_="token-line"):
                            line_content = "".join(
                                token.get_text() for token in span.find_all("span")
                            )
                            lines.append(line_content)

                        code_content = "\n".join(lines)
                        yield f"```{language}\n{code_content}\n```\n\n"
                    else:
                        yield f"`{child.get_text(strip=False)}`"

                elif child.name == "p":
                    yield from get_text(child)
                    yield "\n\n"
                elif child.name == "ul":
                    if "api_reference" in child.attrs:
                        for li in child.find_all("li", recursive=False):
                            yield "> - "
                            yield from get_text(li)
                            yield "\n"
                    else:
                        for li in child.find_all("li", recursive=False):
                            yield "- "
                            yield from get_text(li)
                            yield "\n"
                    yield "\n\n"
                elif child.name == "ol":
                    for i, li in enumerate(child.find_all("li", recursive=False)):
                        yield f"{i + 1}. "
                        yield from get_text(li)
                        yield "\n\n"
                elif child.name == "div" and "tabs-container" in child.attrs.get(
                    "class", [""]
                ):
                    tabs = child.find_all("li", {"role": "tab"})
                    tab_panels = child.find_all("div", {"role": "tabpanel"})
                    for tab, tab_panel in zip(tabs, tab_panels):
                        tab_name = tab.get_text(strip=True)
                        yield f"{tab_name}\n"
                        yield from get_text(tab_panel)
                elif child.name == "table":
                    thead = child.find("thead")
                    header_exists = isinstance(thead, Tag)
                    if header_exists:
                        headers = thead.find_all("th")
                        if headers:
                            yield "| "
                            yield " | ".join(header.get_text() for header in headers)
                            yield " |\n"
                            yield "| "
                            yield " | ".join("----" for _ in headers)
                            yield " |\n"

                    tbody = child.find("tbody")
                    tbody_exists = isinstance(tbody, Tag)
                    if tbody_exists:
                        for row in tbody.find_all("tr"):
                            yield "| "
                            yield " | ".join(
                                cell.get_text(strip=True) for cell in row.find_all("td")
                            )
                            yield " |\n"

                    yield "\n\n"
                elif child.name in ["button"]:
                    continue
                else:
                    yield from get_text(child)

    joined = "".join(get_text(soup))
    return re.sub(r"\n\n+", "\n\n", joined).strip()

In [6]:
loader=load_documents(extractor=partial(langchain_docs_extractor,include_output_cells=True))
docs_with_data_context=loader.load()

El archivo de salida es ahora en formato Markdown, lo que permite visualizarlo en cualquier editor de texto o en GitHub, ofreciendo una estructura de la información más clara y accesible. Esta organización permite realizar cortes de texto con mayor precisión, facilitando así la obtención de información más pertinente y relevante.

In [7]:
Markdown(docs_with_data_context[0].page_content)

# Quickstart

In this quickstart we'll show you how to:

- Get setup with LangChain, LangSmith and LangServe
- Use the most basic and common components of LangChain: prompt templates, models, and output parsers
- Use LangChain Expression Language, the protocol that LangChain is built on and which facilitates component chaining
- Build a simple application with LangChain
- Trace your application with LangSmith
- Serve your application with LangServe

That's a fair amount to cover! Let's dive in.

## Setup​

### Jupyter Notebook​

This guide (and most of the other guides in the documentation) use [Jupyter notebooks](https://jupyter.org/) and assume the reader is as well. Jupyter notebooks are perfect for learning how to work with LLM systems because often times things can go wrong (unexpected output, API down, etc) and going through guides in an interactive environment is a great way to better understand them.

You do not NEED to go through the guide in a Jupyter Notebook, but it is recommended. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation​

To install LangChain run:

Pip
```bash
pip install langchain
```

Conda
```bash
conda install langchain -c conda-forge
```

For more details, see our [Installation guide](/docs/get_started/installation).

### LangSmith​

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

Note that LangSmith is not needed, but it is helpful.
If you do want to use LangSmith, after you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

## Building with LangChain​

LangChain enables building application that connect external sources of data and computation to LLMs.
In this quickstart, we will walk through a few different ways of doing that.
We will start with a simple LLM chain, which just relies on information in the prompt template to respond.
Next, we will build a retrieval chain, which fetches data from a separate database and passes that into the prompt template.
We will then add in chat history, to create a conversation retrieval chain. This allows you interact in a chat manner with this LLM, so it remembers previous questions.
Finally, we will build an agent - which utilizes an LLM to determine whether or not it needs to fetch data to answer questions.
We will cover these at a high level, but there are lot of details to all of these!
We will link to relevant docs.

## LLM Chain​

For this getting started guide, we will provide two options: using OpenAI (a popular model available via API) or using a local open source model.

OpenAI
First we'll need to import the LangChain x OpenAI integration package.

```shell
pip install langchain-openai
```

Accessing the API requires an API key, which you can get by creating an account and heading [here](https://platform.openai.com/account/api-keys). Once we have a key we'll want to set it as an environment variable by running:

```shell
export OPENAI_API_KEY="..."
```

We can then initialize the model:

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
```

If you'd prefer not to set an environment variable you can pass the key in directly via the `openai_api_key` named parameter when initiating the OpenAI LLM class:

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key="...")
```

Local
[Ollama](https://ollama.ai/) allows you to run open-source large language models, such as Llama 2, locally.

First, follow [these instructions](https://github.com/jmorganca/ollama) to set up and run a local Ollama instance:

- [Download](https://ollama.ai/download)
- Fetch a model via `ollama pull llama2`

Then, make sure the Ollama server is running. After that, you can do:

```python
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")
```

Once you've installed and initialized the LLM of your choice, we can try using it!
Let's ask it what LangSmith is - this is something that wasn't present in the training data so it shouldn't have a very good response.

```python
llm.invoke("how can langsmith help with testing?")
```

We can also guide it's response with a prompt template.
Prompt templates are used to convert raw user input to a better input to the LLM.

```python
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])
```

We can now combine these into a simple LLM chain:

```python
chain = prompt | llm 
```

We can now invoke it and ask the same question. It still won't know the answer, but it should respond in a more proper tone for a technical writer!

```python
chain.invoke({"input": "how can langsmith help with testing?"})
```

The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

```python
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
```

We can now add this to the previous chain:

```python
chain = prompt | llm | output_parser
```

We can now invoke it and ask the same question. The answer will now be a string (rather than a ChatMessage).

```python
chain.invoke({"input": "how can langsmith help with testing?"})
```

### Diving Deeper​

We've now successfully set up a basic LLM chain. We only touched on the basics of prompts, models, and output parsers - for a deeper dive into everything mentioned here, see [this section of documentation](/docs/modules/model_io).

## Retrieval Chain​

In order to properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM.
We can do this via _retrieval_.
Retrieval is useful when you have **too much data** to pass to the LLM directly.
You can then use a retriever to fetch only the most relevant pieces and pass those in.

In this process, we will look up relevant documents from a _Retriever_ and then pass them into the prompt.
A Retriever can be backed by anything - a SQL table, the internet, etc - but in this instance we will populate a vector store and use that as a retriever. For more information on vectorstores, see [this documentation](/docs/modules/data_connection/vectorstores).

First, we need to load the data that we want to index. In order to do this, we will use the WebBaseLoader. This requires installing [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/):

```text
```shell
pip install beautifulsoup4
```

After that, we can import and use WebBaseLoader.

```python
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()
```

Next, we need to index it into a vectorstore. This requires a few components, namely an [embedding model](/docs/modules/data_connection/text_embedding) and a [vectorstore](/docs/modules/data_connection/vectorstores).

For embedding models, we once again provide examples for accessing via OpenAI or via local models.

OpenAI
Make sure you have the `langchain_openai` package installed an the appropriate environment variables set (these are the same as needed for the LLM).```python
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
```

Local
Make sure you have Ollama running (same set up as with the LLM).

```python
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()
```

Now, we can use this embedding model to ingest documents into a vectorstore.
We will use a simple local vectorstore, [FAISS](/docs/integrations/vectorstores/faiss), for simplicity's sake.

First we need to install the required packages for that:

```shell
pip install faiss-cpu
```

Then we can build our index:

```python
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
```

Now that we have this data indexed in a vectorstore, we will create a retrieval chain.
This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

```python
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)
```

If we wanted to, we could run this ourselves by passing in documents directly:

```python
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})
```

However, we want the documents to first come from the retriever we just set up.
That way, for a given question we can use the retriever to dynamically select the most relevant documents and pass those in.

```python
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
```

We can now invoke this chain. This returns a dictionary - the response from the LLM is in the `answer` key

```python
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...
```

This answer should be much more accurate!

### Diving Deeper​

We've now successfully set up a basic retrieval chain. We only touched on the basics of retrieval - for a deeper dive into everything mentioned here, see [this section of documentation](/docs/modules/data_connection).

## Conversation Retrieval Chain​

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the `create_retrieval_chain` function, but we need to change two things:

1. The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.

2. The final LLM chain should likewise take the whole history into account

**Updating Retrieval**

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (`input`) and the conversation history (`chat_history`) and use an LLM to generate a search query.

```python
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)
```

We can test this out by passing in an instance where the user is asking a follow up question.

```python
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
```

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

```python
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)
```

We can now test this out end-to-end:

```python
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
```

We can see that this gives a coherent answer - we've successfully turned our retrieval chain into a chatbot!

## Agent​

We've so far create examples of chains - where each step is known ahead of time.
The final thing we will create is an agent - where the LLM decides what steps to take.

**NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.**

One of the first things to do when building an agent is to decide what tools it should have access to.
For this example, we will give the agent access two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith

2. A search tool. This will let it easily answer questions that require up to date information.

First, let's set up a tool for the retriever we just created:

```python
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
```

The search tool that we will use is [Tavily](/docs/integrations/retrievers/tavily). This will require an API key (they have generous free tier). After creating it on their platform, you need to set it as an environment variable:

```shell
export TAVILY_API_KEY=...
```

If you do not want to set up an API key, you can skip creating this tool.

```python
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
```

We can now create a list of the tools we want to work with:

```python
tools = [retriever_tool, search]
```

Now that we have the tools, we can create an agent to use them. We will go over this pretty quickly - for a deeper dive into what exactly is going on, check out the [Agent's Getting Started documentation](/docs/modules/agents)

Install langchain hub first

```bash
pip install langchainhub
```

Now we can use it to get a predefined prompt

```python
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
```

We can now invoke the agent and see how it responds! We can ask it questions about LangSmith:

```python
agent_executor.invoke({"input": "how can langsmith help with testing?"})
```

We can ask it about the weather:

```python
agent_executor.invoke({"input": "what is the weather in SF?"})
```

We can have conversations with it:

```python
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})
```

### Diving Deeper​

We've now successfully set up a basic agent. We only touched on the basics of agents - for a deeper dive into everything mentioned here, see [this section of documentation](/docs/modules/agents).

## Serving with LangServe​

Now that we've built an application, we need to serve it. That's where LangServe comes in.
LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with:

```bash
pip install "langserve[all]"
```

### Server​

To create a server for our application we'll make a `serve.py` file. This will contain our logic for serving our application. It consists of three things:

1. The definition of our chain that we just built above

2. Our FastAPI app

3. A definition of a route from which to serve the chain, which is done with `langserve.add_routes`

```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes

# 1. Load Retriever
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 2. Create Tools
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
search = TavilySearchResults()
tools = [retriever_tool, search]

# 3. Create Agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

# We need to add these input/output schemas because the current AgentExecutor
# is lacking in schemas.

class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
        ...,
        extra={"widget": {"type": "chat", "input": "location"}},
    )

class Output(BaseModel):
    output: str

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

And that's it! If we execute this file:

```bash
python serve.py
```

we should see our chain being served at localhost:8000.

### Playground​

Every LangServe service comes with a simple built-in UI for configuring and invoking the application with streaming output and visibility into intermediate steps.
Head to http://localhost:8000/agent/playground/ to try it out! Pass in the same question as before - "how can langsmith help with testing?" - and it should respond same as before.

### Client​

Now let's set up a client for programmatically interacting with our service. We can easily do this with the `[langserve.RemoteRunnable](/docs/langserve#client)`.
Using this, we can interact with the served chain as if it were running client-side.

```python
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/agent/")
remote_chain.invoke({"input": "how can langsmith help with testing?"})
```

To learn more about the many other features of LangServe [head here](/docs/langserve).

## Next steps​

We've touched on how to build an application with LangChain, how to trace it with LangSmith, and how to serve it with LangServe.
There are a lot more features in all three of these than we can cover here.
To continue on your journey, we recommend you read the following (in order):

- All of these features are backed by [LangChain Expression Language (LCEL)](/docs/expression_language) - a way to chain these components together. Check out that documentation to better understand how to create custom chains.
- [Model IO](/docs/modules/model_io) covers more details of prompts, LLMs, and output parsers.
- [Retrieval](/docs/modules/data_connection) covers more details of everything related to retrieval
- [Agents](/docs/modules/agents) covers details of everything related to agents
- Explore common [end-to-end use cases](/docs/use_cases/) and [template applications](/docs/templates)
- [Read up on LangSmith](/docs/langsmith/), the platform for debugging, testing, monitoring and more
- Learn more about serving your applications with [LangServe](/docs/langserve)

## PDF / DOCX / DOC

### Dataset de prueba

En este ejemplo, vamos a emplear algunos archivos de muestra proporcionados por [Docugami](https://www.docugami.com/). Dichos archivos representan el producto de la extracción de texto de documentos auténticos, en particular, de archivos PDF relativos a contratos de arrendamiento comercial.

In [20]:
lease_data_dir = pathlib.Path("./data/docugami/commercial_lease")
lease_files = list(lease_data_dir.glob("*"))
lease_files

[PosixPath('data/docugami/commercial_lease/TruTone Lane 5.xml'),
 PosixPath('data/docugami/commercial_lease/TruTone Lane 3.xml'),
 PosixPath('data/docugami/commercial_lease/TruTone Lane 2.xml'),
 PosixPath('data/docugami/commercial_lease/TruTone Lane 6.xml'),
 PosixPath('data/docugami/commercial_lease/TruTone Lane 1.xml'),
 PosixPath('data/docugami/commercial_lease/TruTone Lane 4.xml')]

Ahora, carguemos los documentos de muestra y veamos qué propiedades tienen.

In [22]:
loader=DocugamiLoader(docset_id=None,access_token=None,document_ids=None,file_paths=lease_files)

lease_docs=loader.load()

f"Loaded {len(lease_docs)} documents"

'Loaded 1130 documents'

La metadata obtenida del documento incluye los siguientes elementos:

- `id`, `source_id` y `name`: Estos campos identifican de manera unívoca al documento y al fragmento de texto que se ha extraído de él.
- `xpath`: Es el `XPath` correspondiente dentro de la representación XML del documento. Se refiere específicamente al fragmento extraído. Este campo es útil para referenciar directamente las citas del fragmento real dentro del documento XML.
- `structure`: Incluye los atributos estructurales del fragmento, tales como `p`, `h1`, `div`, `table`, `td`, entre otros. Es útil para filtrar ciertos tipos de fragmentos, en caso de que el usuario los requiera.
- `tag`: Representa la etiqueta semántica para el fragmento. Se genera utilizando diversas técnicas, tanto generativas como extractivas, para determinar el significado del fragmento en cuestión.

In [24]:
lease_docs[0].metadata

{'xpath': '/docset:RIDERTOLEASE-section/dg:chunk',
 'id': 'f807c0113ccbb1a2a82e9eedbbe0c527',
 'name': 'TruTone Lane 5.xml',
 'source': 'TruTone Lane 5.xml',
 'structure': 'h1 p',
 'tag': 'chunk IncorporationofTerms'}

`Docugami` también posee la capacidad de asistir en la extracción de metadatos específicos para cada `chunk` o fragmento de nuestros documentos. A continuación, se presenta un ejemplo de cómo se extraen y representan estos metadatos:

```json
{
    'xpath': '/docset:OFFICELEASEAGREEMENT-section/docset:OFFICELEASEAGREEMENT/docset:LeaseParties',
    'id': 'v1bvgaozfkak',
    'source': 'TruTone Lane 2.docx',
    'structure': 'p',
    'tag': 'LeaseParties',
    'Lease Date': 'April 24 \n\n ,',
    'Landlord': 'BUBBA CENTER PARTNERSHIP',
    'Tenant': 'Truetone Lane LLC',
    'Lease Parties': 'Este ACUERDO DE ARRENDAMIENTO DE OFICINA (el "Contrato") es celebrado por y entre BUBBA CENTER PARTNERSHIP ("Arrendador"), y Truetone Lane LLC, una compañía de responsabilidad limitada de Delaware ("Arrendatario").'
}
```

Los metadatos adicionales, como los mostrados arriba, pueden ser extremadamente útiles cuando se implementan `self-retrievers`, los cuales serán explorados adetalle más adelante.

### Cargar tus documentos

Si prefieres utilizar tus propios documentos, puedes cargarlos a través de la interfaz gráfica de [Docugami](https://www.docugami.com/). Una vez cargados, necesitarás asignar cada uno a un `docset`. Un `docset` es un conjunto de documentos que presentan una estructura análoga. Por ejemplo, todos los contratos de arrendamiento comercial por lo general poseen estructuras similares, por lo que pueden ser agrupados en un único `docset`.

Después de crear tu `docset`, los documentos cargados serán procesados y estarán disponibles para su acceso mediante la API de `Docugami`.

Para recuperar los `ids` de tus documentos y de sus correspondientes `docsets`, puedes ejecutar el siguiente comando:

```bash
curl --header "Authorization: Bearer {YOUR_DOCUGAMI_TOKEN}" \
  https://api.docugami.com/v1preview1/documents
```

Este comando te facilitará el acceso a la información relevante, optimizando así la administración y organización de tus documentos dentro de `Docugami`.

Una vez hayas extraído los `ids` de tus documentos o de los `docsets`, podrás emplearlos para acceder a la información de tus documentos utilizando el `DocugamiLoader` de `Langchain`. Esto te permitirá manipular y gestionar tus documentos dentro de tu aplicación.

In [ ]:
loader = DocugamiLoader(
    docset_id="",
    document_ids=None,
    file_paths=None,
)

papers_docs = loader.load()

In [ ]:
lost_in_the_middle_paper_docs = [
    doc for doc in papers_docs if doc.metadata["source"] == ""
]
for doc in lost_in_the_middle_paper_docs:
    print(doc.metadata["tag"])